<a href="https://colab.research.google.com/github/reaganmathai/DiD-San-Francisco-Rent-Analysis/blob/main/Stock_Price_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas matplotlib

In [5]:
import os
import pandas as pd

# Define paths
current_directory = os.getcwd()
input_file_path = os.path.join(current_directory, 'FinalDatabase.csv')  # Input file location
output_file_path = os.path.join(current_directory, 'SentimentDispersion.csv')  # Output file location

# Load the dataset
df = pd.read_csv('/content/FinalStockDatabase.csv')

# Convert date columns to datetime format, correcting the format to match your data
df['date'] = pd.to_datetime(df['date'], format='%d%b%Y')  # Corrected format here
df['rdq'] = pd.to_datetime(df['rdq'], format='%d%b%Y')    # Corrected format here

# Define the period before the earnings announcement date (e.g., B0 = 10 days, B1 = 5 days before)
B0 = 10  # Start of the period (10 days before)
B1 = 5   # End of the period (5 days before)

# Prepare a list to collect results
results = []

# Iterate over each firm
for firm_id in df['firm_id'].unique():
    firm_data = df[df['firm_id'] == firm_id]

    # Iterate over each earnings announcement date for the firm
    for rdq in firm_data['rdq'].dropna().unique():  # Ensure rdq is not NaT
        # Select the data from B0 to B1 days before the announcement
        period_start = rdq - pd.Timedelta(days=B0)
        period_end = rdq - pd.Timedelta(days=B1)
        period_data = firm_data[(firm_data['date'] >= period_start) & (firm_data['date'] < period_end)]

        # Calculate sentiment range and standard deviation, ignoring NaN values
        if not period_data.empty:
            max_sentiment = period_data['sentiment'].max(skipna=True)
            min_sentiment = period_data['sentiment'].min(skipna=True)
            sentiment_range = max_sentiment - min_sentiment if pd.notnull(max_sentiment) and pd.notnull(min_sentiment) else None
            sentiment_std = period_data['sentiment'].std(skipna=True)
        else:
            sentiment_range = None
            sentiment_std = None

        # Collect results
        results.append({
            'firm_id': firm_id,
            'rdq': rdq,
            'sentiment_range': sentiment_range,
            'sentiment_std': sentiment_std
        })

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Save the results to a CSV file
results_df.to_csv(output_file_path, index=False)

print(f"Sentiment dispersion data saved to {output_file_path}")

<ipython-input-5-b710e4e429fd>:10: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/FinalStockDatabase.csv')


Sentiment dispersion data saved to /content/SentimentDispersion.csv


In [8]:
import matplotlib.pyplot as plt
from pandas.tseries.offsets import BDay

data = pd.read_csv('/content/FinalStockDatabase.csv')

# Define your constants for business days before and after the report date (RDQ)
D0 = 5  # Days before the earnings announcement
D1 = 5  # Days after the earnings announcement

# Iterate over each unique firm

# Convert 'date' column to datetime
data['date'] = pd.to_datetime(data['date'], errors='coerce')

# Drop rows where 'date' or 'price' is NaN
data.dropna(subset=['date', 'price'], inplace=True)

# Ensure data is sorted by date for each firm_id
data.sort_values(by=['firm_id', 'date'], inplace=True)

# Calculate daily returns for each firm
data['daily_return'] = data.groupby('firm_id')['price'].pct_change()

# Calculate the average daily return for each firm
average_daily_returns = data.groupby('firm_id')['daily_return'].mean()

# Convert the average daily returns to a DataFrame and reset the index for easier reading
average_daily_returns_df = average_daily_returns.reset_index()

# Give the columns meaningful names
average_daily_returns_df.columns = ['firm_id', 'average_daily_return']

# Display the average daily returns for each firm
print(average_daily_returns_df)

<ipython-input-8-7c877321c36c>:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/FinalStockDatabase.csv')


      firm_id  average_daily_return
0           3              0.001575
1           4              0.000968
2          11              0.001006
3          14              0.010243
4          15              0.000110
...       ...                   ...
3023    10838              0.000032
3024    10839             -0.000006
3025    10842              0.001094
3026    10847              0.001093
3027    10848              0.000972

[3028 rows x 2 columns]


# 4/1

In [10]:
import os
import pandas as pd
from pandas.tseries.offsets import BDay

def calculate_adjusted_daily_returns(input_file_path, D0, D1, output_file_path):
    # Load and prepare the data
    data = pd.read_csv('/content/FinalStockDatabase.csv')  # Make sure you're loading the correct file. Previously, it was hardcoded to '/content/SentimentDispersion.csv'.

    # First, check the column names to ensure 'date' and 'rdq' are present
    print(data.columns)  # This will print out all column names. Ensure there's 'date' and 'rdq'.

    # Ensuring correct datetime format and coercing errors
    # Make sure your CSV file has a 'date' and 'rdq' column. If the column names are different, you'll need to adjust them here.
    data['date'] = pd.to_datetime(data['date'], errors='coerce', format='%d%b%Y')
    data['rdq'] = pd.to_datetime(data['rdq'], errors='coerce', format='%d%b%Y')

    # Dropping rows where either date or price is NaN to ensure clean data
    data.dropna(subset=['date', 'price', 'rdq'], inplace=True)

    results = []

    for firm_id in data['firm_id'].unique():
        firm_data = data[data['firm_id'] == firm_id].sort_values('date')

        for rdq in firm_data['rdq'].unique():
            rdq_date = pd.to_datetime(rdq)
            start_date = rdq_date - BDay(D0)
            end_date = rdq_date + BDay(D1)

            adjusted_start_date = firm_data[(firm_data['date'] <= start_date)].dropna(subset=['price']).iloc[-1]['date'] if not firm_data[(firm_data['date'] <= start_date)].empty else None
            adjusted_end_date = firm_data[(firm_data['date'] >= end_date)].dropna(subset=['price']).iloc[0]['date'] if not firm_data[(firm_data['date'] >= end_date)].empty else None

            if adjusted_start_date is not None and adjusted_end_date is not None:
                price_start = firm_data[firm_data['date'] == adjusted_start_date]['price'].values[0]
                price_end = firm_data[firm_data['date'] == adjusted_end_date]['price'].values[0]
                days = (adjusted_end_date - adjusted_start_date).days

                stock_return = ((price_end - price_start) / price_start) / days if days > 0 else 0

                results.append({
                    'firm_id': firm_id,
                    'rdq': rdq,
                    'daily_return': stock_return,
                    'adjusted_start_date': adjusted_start_date.strftime('%Y-%m-%d'),
                    'adjusted_end_date': adjusted_end_date.strftime('%Y-%m-%d'),
                    'days': days
                })

    # Converting results into a DataFrame and saving to CSV
    results_df = pd.DataFrame(results)
    results_df.to_csv(output_file_path, index=False)
    print(f"Results saved to {output_file_path}")

# Ensure you have the correct file paths
current_directory = os.getcwd()
input_file_path = os.path.join(current_directory, 'FinalDatabase.csv')  # Ensure this matches the file you intend to process
output_file_path = os.path.join(current_directory, 'AdjustedDailyReturns.csv')
D0 = 2  # Days before the earnings announcement
D1 = 5  # Days after the earnings announcement

# Before running the function, make sure you've corrected any discrepancies in column names or file paths.
calculate_adjusted_daily_returns(input_file_path, D0, D1, output_file_path)

<ipython-input-10-9d609a8519f8>:7: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/FinalStockDatabase.csv')  # Make sure you're loading the correct file. Previously, it was hardcoded to '/content/SentimentDispersion.csv'.


Index(['date', 'price', 'firm_id', 'sentiment', 'Systematic_Risk',
       'Idiosyncratic_Risk', 'rdq', 'Interest_Coverage', 'Dividend_Payer',
       'RF', 'mktcap', 'Size'],
      dtype='object')
Results saved to /content/AdjustedDailyReturns.csv


In [11]:
import pandas as pd

# Load both datasets
sentiment_df = pd.read_csv('SentimentDispersion.csv')
returns_df = pd.read_csv('AdjustedDailyReturns.csv')

# Merge the datasets on 'firm_id' and 'rdq'
merged_df = pd.merge(sentiment_df, returns_df, on=['firm_id', 'rdq'])

# Drop rows with missing values
cleaned_df = merged_df.dropna()

# Optionally, inspect the cleaned data
print(cleaned_df.head())

# Save the cleaned dataframe to a new CSV file
cleaned_df.to_csv('Portfolio.csv', index=False)

     firm_id         rdq  sentiment_range  sentiment_std  daily_return  \
101      210  2010-05-18          0.04400       0.031113     -0.010510   
479      891  2010-04-22          0.00565       0.003995     -0.001903   
481      891  2010-10-21          0.00380       0.002687      0.005721   
565     1016  2010-08-06          0.00510       0.002944     -0.009301   
566     1016  2010-11-05          1.79740       0.851592     -0.004488   

    adjusted_start_date adjusted_end_date  days  
101          2010-05-14        2010-05-25    11  
479          2010-04-20        2010-04-29     9  
481          2010-10-19        2010-10-28     9  
565          2010-08-04        2010-08-13     9  
566          2010-11-03        2010-11-12     9  


In [12]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/content/FinalStockDatabase.csv')

# Convert 'date' and 'rdq' columns to datetime format
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['rdq'] = pd.to_datetime(df['rdq'], errors='coerce')

<ipython-input-12-782d91f4e144>:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/FinalStockDatabase.csv')


In [13]:
from pandas.tseries.offsets import BDay

# Assume B0 and B1 based on your benchmark scenario
B0, B1 = 9, 2

# Prepare a DataFrame to store results
results = []

for firm_id in df['firm_id'].unique():
    firm_data = df[df['firm_id'] == firm_id].sort_values(by='date')
    for rdq in firm_data['rdq'].unique():
        period_start = rdq - BDay(B0)
        period_end = rdq - BDay(B1)
        period_data = firm_data[(firm_data['date'] >= period_start) & (firm_data['date'] <= period_end)]

        sentiment_range = period_data['sentiment'].max() - period_data['sentiment'].min() if not period_data.empty else None
        sentiment_std = period_data['sentiment'].std() if not period_data.empty else None

        results.append({
            'firm_id': firm_id,
            'rdq': rdq,
            'sentiment_range': sentiment_range,
            'sentiment_std': sentiment_std
        })

results_df = pd.DataFrame(results)

In [14]:
D0, D1 = 2, 5

returns = []

for firm_id in df['firm_id'].unique():
    firm_data = df[df['firm_id'] == firm_id].sort_values(by='date')
    for rdq in firm_data['rdq'].unique():
        start_date = rdq - BDay(D0)
        end_date = rdq + BDay(D1)

        # Ensure the dates are within the data's range
        if firm_data['date'].min() <= start_date <= firm_data['date'].max() and firm_data['date'].min() <= end_date <= firm_data['date'].max():
            start_price = firm_data[firm_data['date'] == start_date]['price'].iloc[0]
            end_price = firm_data[firm_data['date'] == end_date]['price'].iloc[0]
            stock_return = (end_price - start_price) / start_price

            returns.append({
                'firm_id': firm_id,
                'rdq': rdq,
                'stock_return': stock_return
            })

returns_df = pd.DataFrame(returns)

In [17]:
# Drop rows where 'sentiment_range' is NaN to ensure clean data for processing
cleaned_df = merged_df.dropna(subset=['sentiment_range'])

# Determine the quantiles including NaN handling
quantiles = cleaned_df['sentiment_range'].quantile([0.2, 0.4, 0.6, 0.8]).tolist()

# Define a function to manually categorize data into quintiles
def categorize_into_quintiles(value, boundaries):
    if pd.isna(value):
        return np.nan
    for i, boundary in enumerate(boundaries):
        if value <= boundary:
            return i
    return len(boundaries)

# Apply the categorization
cleaned_df['quintile'] = cleaned_df['sentiment_range'].apply(lambda x: categorize_into_quintiles(x, quantiles))

# Now, calculate portfolio returns based on these manually categorized quintiles
portfolio_returns = cleaned_df.groupby('quintile')['stock_return'].mean().reset_index()

print(portfolio_returns)

   quintile  stock_return
0         0     -0.005457
1         2     -0.022080
2         3     -0.013683
3         4     -0.013243


<ipython-input-17-e8aeab6c127d>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['quintile'] = cleaned_df['sentiment_range'].apply(lambda x: categorize_into_quintiles(x, quantiles))
